In [1]:
%run imports_and_functions.ipynb

from dbfread import DBF

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# income data files come in .dbf, must install this to read from python

# !pip install dbfread

### Creating Final Businesses Dataframe within City Radius

In [2]:
df = pd.read_csv('LasVegasBusinessesUpdated.csv', sep=';', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14880 entries, 0 to 14879
Data columns (total 13 columns):
Unnamed: 0       14880 non-null int64
alias            14880 non-null object
attributes       0 non-null float64
categories       14880 non-null object
coordinates      14880 non-null object
distance         14880 non-null float64
latitude         14879 non-null float64
longitude        14879 non-null float64
main_category    14880 non-null object
name             14880 non-null object
price            6267 non-null object
rating           14880 non-null float64
review_count     14880 non-null int64
dtypes: float64(5), int64(2), object(6)
memory usage: 1.5+ MB


In [3]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask1 = dups[['latitude','longitude','alias']].duplicated(keep='first')
    mask2 = dups[['latitude','longitude','alias']].duplicated(keep=False)
    mask = mask2 & ~mask1
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)
        
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0
    
def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

In [4]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape

(14880, 13)


(14877, 13)

In [5]:
df['price'] = df['price'].map(convertpricecolumn)
df['price'].value_counts()

0    8613
2    3385
1    1794
3     745
4     340
Name: price, dtype: int64

In [6]:
# df['main_category'] = df['categories'].map(get_main_category)
df['main_category'].value_counts()

shopping              2800
homeservices          2708
restaurants           2456
automotive             799
beautysvc              787
food                   682
arts                   664
health                 653
localservices          607
eventservices          547
nightlife              518
professional           368
active                 324
hotelstravel           320
financialservices      237
realestate             111
education              108
pets                    95
publicservicesgovt      32
massmedia               31
religiousorgs           30
Name: main_category, dtype: int64

In [7]:
# SET FOR EACH CITY: maxradius, testlat, testlong
maxradius = 7000
testlat = 36.120743
testlong = -115.192017 # Las Vegas dataset

In [8]:
df['distance'] = calcDistances(testlat, testlong, df)

df[df['distance']<maxradius].shape

(11586, 13)

In [10]:
df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

,Unnamed: 0,alias,attributes,categories,coordinates,distance,latitude,longitude,main_category,name,price,rating,review_count
988,1010,lindy-bakers-super-intuition-henderson-2,NaN,"[{'alias': 'lifecoach', 'title': 'Life Coach'}...","{'latitude': 35.9999694824219, 'longitude': -1...",15900.059981,35.999969,-115.097038,arts,Lindy Baker's Super Intuition,0,5.0,9
989,1011,love-spells-by-charlotte-russell-henderson,NaN,"[{'alias': 'psychic_astrology', 'title': 'Supe...","{'latitude': 35.9999694824219, 'longitude': -1...",15900.059981,35.999969,-115.097038,arts,Love Spells by Charlotte Russell,0,5.0,1
6546,7279,ridpro-pest-control-las-vegas,NaN,"[{'alias': 'pest_control', 'title': 'Pest Cont...","{'latitude': 36.02138, 'longitude': -115.17166}",11176.816079,36.021380,-115.171660,localservices,RidPro Pest Control,0,5.0,1
12633,15683,southwest-gas-corporation-las-vegas,NaN,"[{'alias': 'utilities', 'title': 'Utilities'}]","{'latitude': 36.02138, 'longitude': -115.17166}",11176.816079,36.021380,-115.171660,homeservices,Southwest Gas Corporation,0,1.5,54
12634,15684,leak-point-las-vegas-4,NaN,"[{'alias': 'plumbing', 'title': 'Plumbing'}]","{'latitude': 36.02138, 'longitude': -115.17166}",11176.816079,36.021380,-115.171660,homeservices,Leak Point,0,2.5,4
3141,3288,marc-fragos-photography-las-vegas,NaN,"[{'alias': 'photographers', 'title': 'Photogra...","{'latitude': 36.02138, 'longitude': -115.17166}",11176.816079,36.021380,-115.171660,eventservices,Marc Fragos Photography,0,5.0,1
6545,7278,trauma-intervention-program-of-southern-nevada...,NaN,"[{'alias': 'nonprofit', 'title': 'Community Se...","{'latitude': 36.02138, 'longitude': -115.17166}",11176.816079,36.021380,-115.171660,localservices,Trauma Intervention Program of Southern Nevada,0,5.0,1
12675,15726,clear-las-vegas-4,NaN,"[{'alias': 'isps', 'title': 'Internet Service ...","{'latitude': 36.03884, 'longitude': -115.11901}",11217.605813,36.038840,-115.119010,homeservices,Clear,0,1.0,1
12676,15727,premier-pools-and-spas-las-vegas-3,NaN,"[{'alias': 'poolservice', 'title': 'Pool & Hot...","{'latitude': 36.03884, 'longitude': -115.11901}",11217.605813,36.038840,-115.119010,homeservices,Premier Pools & Spas,0,2.0,14
12677,15728,viewpointe-executive-suites-las-vegas-2,NaN,"[{'alias': 'sharedofficespaces', 'title': 'Sha...","{'latitude': 36.038875579834, 'longitude': -11...",11214.760853,36.038876,-115.119003,homeservices,Viewpointe Executive Suites,0,4.0,7


In [11]:
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)
df.shape

(11586, 13)

In [12]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = df['longitude']

df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
df = df.sort_index(level = [0,1])

In [13]:
df.head()

Unnamed: 0  \
lat_index long_index                
36.058    -115.183435       18750   
36.059    -115.183514       15819   
36.060    -115.211913       13870   
          -115.210210       13899   
          -115.210210       13900   

                                                              alias  \
lat_index long_index                                                  
36.058    -115.183435  best-mattress-liquidation-center-las-vegas-4   
36.059    -115.183514                   ies-residential-las-vegas-2   
36.060    -115.211913                           switchnap-las-vegas   
          -115.210210        martin-harris-construction-las-vegas-5   
          -115.210210                   switch-supernap-7-las-vegas   

                       attributes  \
lat_index long_index                
36.058    -115.183435         NaN   
36.059    -115.183514         NaN   
36.060    -115.211913         NaN   
          -115.210210         NaN   
          -115.210210         NaN   

                                                              categories  \
lat_index long_index                                                       
36.058    -115.183435  [{'alias': 'mattresses', 'title': 'Mattresses'...   
36.059    -115.183514  [{'alias': 'excavationservices', 'title': 'Exc...   
36.060    -115.211913  [{'alias': 'isps', 'title': 'Internet Service ...   
          -115.210210  [{'alias': 'contractors', 'title': 'Contractor...   
          -115.210210  [{'alias': 'telecommunications', 'title': 'Tel...   

                                                             coordinates  \
lat_index long_index                                                       
36.058    -115.183435  {'latitude': 36.05846, 'longitude': -115.1834351}   
36.059    -115.183514  {'latitude': 36.0593128885251, 'longitude': -1...   
36.060    -115.211913  {'latitude': 36.0603091231611, 'longitude': -1...   
          -115.210210    {'latitude': 36.05989, 'longitude': -115.21021}   
          -115.210210    {'latitude': 36.05989, 'longitude': -115.21021}   

                          distance   latitude   longitude main_category  \
lat_index long_index                                                      
36.058    -115.183435  6954.047910  36.058460 -115.183435      shopping   
36.059    -115.183514  6859.204503  36.059313 -115.183514  homeservices   
36.060    -115.211913  6941.060362  36.060309 -115.211913  homeservices   
          -115.210210  6948.238755  36.059890 -115.210210  homeservices   
          -115.210210  6948.238755  36.059890 -115.210210  homeservices   

                                                   name  price  rating  \
lat_index long_index                                                     
36.058    -115.183435  Best Mattress Liquidation Center      2     4.0   
36.059    -115.183514                   IES Residential      0     1.0   
36.060    -115.211913                         SwitchNAP      0     1.0   
          -115.210210        Martin Harris Construction      0     1.0   
          -115.210210                 Switch SUPERNAP 7      0     5.0   

                       review_count  lat_index  long_index  
lat_index long_index                                        
36.058    -115.183435            22     36.058 -115.183435  
36.059    -115.183514             1     36.059 -115.183514  
36.060    -115.211913             1     36.060 -115.211913  
          -115.210210             1     36.060 -115.210210  
          -115.210210             1     36.060 -115.210210

In [14]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat)])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [15]:
df['alias_cats'] = df.apply(categories_to_alias2, axis=1)

In [17]:

df.to_csv('LasVegasBusinessesFinalUpdated.csv',sep=';',header=True,index=True,compression='gzip')

In [16]:
df['alias_cats']

lat_index  long_index 
36.058     -115.183435                               2_furniture 2_shopping
36.059     -115.183514                                       0_homeservices
36.060     -115.211913                        0_homeservices 0_professional
           -115.210210                         0_homeservices 0_contractors
           -115.210210        0_localservices 0_homeservices 0_professional
           -115.182419                                       0_homeservices
           -115.182402       0_automotive 0_auto 0_carwash 0_auto_detailing
           -115.178790                                       0_homeservices
           -115.171173                          2_newamerican 2_restaurants
36.061     -115.177360                         0_homeservices 0_contractors
           -115.176767                 2_shopping 2_womenscloth 2_menscloth
           -115.176671                       1_food 1_burgers 1_restaurants
36.062     -115.210213                                       0_ho

### Census Block Population and Income to X_train Coordinates DataFrame

In [22]:
# table = DBF('tl_2014_04_tabblock10.dbf')
# type(table)

dbfread.dbf.DBF

In [ ]:
# df_coords = pd.DataFrame(iter(table))
# df_coords.head()

In [23]:
# Already in csv form:
# THE FOLLOWING 2 RAW FILES CAN BE FOUND IN LAS VEGAS DATA FOLDER
df_coords = pd.read_csv('ClarkCountyCensusBlocks.csv')

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,04,001,970502,1599,040019705021599,Block 1599,G5040,R,,,S,0,251271,+33.9097858,-109.4197413
1,04,011,960100,1275,040119601001275,Block 1275,G5040,R,,,S,0,8809,+33.1205764,-109.2852833
2,04,023,966101,1103,040239661011103,Block 1103,G5040,R,,,S,3013295,1901,+31.6172066,-111.0172819
3,04,023,966105,1080,040239661051080,Block 1080,G5040,R,,,S,576618,918,+31.3361979,-110.9826473
4,04,023,966105,1068,040239661051068,Block 1068,G5040,R,,,S,36049198,11450,+31.4063471,-111.1182378


In [24]:
df_coords = df_coords[['GEOID10','INTPTLAT10','INTPTLON10','UR10','ALAND10','AWATER10']]
df_coords.to_csv('LasVegasCoords.csv',sep=';',header=True,index=True,compression='gzip')

In [25]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])

def matchincome(geoid) :
    try : return blockgroupincome.loc[geoid,'income']
    except : return 0

def matchpop(geoid) :
    try : return blockgrouppop.loc[geoid,'population']
    except : return 0

In [26]:
# table = DBF('tabblock2010_32_pophu.dbf')

# populations = pd.DataFrame(iter(table))
# populations.head()

# Already in csv form:
populations = pd.read_csv('tabblock2010_32_pophu.csv')

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10
0,04,027,011107,1059,040270111071059,N,0,0
1,04,027,011107,2002,040270111072002,N,0,0
2,04,027,011107,2045,040270111072045,N,0,0
3,04,027,012100,1437,040270121001437,N,0,0
4,04,027,010914,2037,040270109142037,N,0,0


In [27]:
populations = populations[['BLOCKID10','POP10']]

In [28]:
populations.columns = ['GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations['GEO.id2'] = populations['GEO.id2'].map(int)
populations.set_index(['GEO.id2'],inplace=True,drop=False)
populations.head()

,GEO.id2,population,blockgroup
GEO.id2,,,
40270111071059,40270111071059,0,40270111071
40270111072002,40270111072002,0,40270111072
40270111072045,40270111072045,0,40270111072
40270121001437,40270121001437,0,40270121001
40270109142037,40270109142037,0,40270109142


In [29]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241666 entries, 40270111071059 to 40250016033039
Data columns (total 3 columns):
GEO.id2       241666 non-null int64
population    241666 non-null int64
blockgroup    241666 non-null int64
dtypes: int64(3)
memory usage: 7.4 MB


In [30]:
blockgroupincome = pd.read_csv('ACS_17_ArizonaIncome.csv')
blockgroupincome.head()

,GEO.id2,population,income
0,Id2,Estimate,NaN
1,40130101011,54499,1818.0
2,40130101012,43901,1758.0
3,40130101013,72350,1339.0
4,40130101021,78229,978.0


In [31]:
blockgroupincome.drop(0, axis=0, inplace=True)
blockgroupincome['GEO.id2'] = blockgroupincome['GEO.id2'].map(int)
blockgroupincome.set_index('GEO.id2', inplace=True)
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2503 entries, 40130101011 to 40139807001
Data columns (total 2 columns):
population    2503 non-null object
income        2503 non-null float64
dtypes: float64(1), object(1)
memory usage: 58.7+ KB


In [32]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations['income'].value_counts()

0.0       169406
910.0        810
532.0        787
978.0        672
3289.0       639
2206.0       635
2921.0       622
5942.0       531
4068.0       487
1218.0       429
1731.0       416
1514.0       327
1151.0       321
1085.0       304
2447.0       269
3740.0       260
3332.0       256
1923.0       239
916.0        228
1359.0       227
1775.0       225
1709.0       216
1161.0       206
1506.0       205
2573.0       203
1362.0       199
1630.0       195
2503.0       194
2961.0       189
1804.0       185
3857.0       182
1267.0       177
2903.0       171
4415.0       171
1596.0       170
1123.0       158
1991.0       156
1600.0       154
1822.0       153
1794.0       153
4499.0       152
2092.0       152
7602.0       150
1407.0       150
1862.0       147
1480.0       147
2185.0       147
1811.0       144
1758.0       142
1645.0       141
1052.0       140
3791.0       134
1856.0       133
3133.0       133
1785.0       132
1196.0       128
1507.0       127
1809.0       126
1339.0       1

In [33]:
blockgrouppop = blockgroupincome

populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
populations['blockgrouppop'].value_counts()

0         168987
17326        787
14746        753
78229        661
18185        639
22581        622
31203        573
3880         531
33868        487
-            419
32150        416
20328        392
18930        301
33053        282
21180        278
19604        269
21896        260
47731        256
21618        239
18767        228
30785        225
16644        203
24263        196
30453        182
18186        172
46435        171
12395        171
31979        171
15586        160
26170        157
39747        154
43373        152
29197        150
24003        150
26288        145
25530        138
12132        134
23799        133
112222       126
20814        122
17091        119
12895        116
43901        114
35941        113
33372        112
28170        111
35455        111
20964        109
25307        108
38521        107
21718        105
36011        104
27142        101
15428        100
32154        100
54499         98
38691         98
34027         97
18003         

### X_train Coordinates DataFrame

In [18]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [19]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    mask = df['income'].astype(str)=='-'
    df.drop(df[mask].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [37]:
blocks = pd.read_csv('LasVegasCoords.csv',sep=';',compression='gzip')
blocks.head()

,Unnamed: 0,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,0,40019705021599,33.909786,-109.419741,R,0,251271
1,1,40119601001275,33.120576,-109.285283,R,0,8809
2,2,40239661011103,31.617207,-111.017282,R,3013295,1901
3,3,40239661051080,31.336198,-110.982647,R,576618,918
4,4,40239661051068,31.406347,-111.118238,R,36049198,11450


In [38]:
blocks.drop('Unnamed: 0', axis=1, inplace=True)
blocks.columns = ['GEO.id2','latitude','longitude','urban','landarea','waterarea']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks['GEO.id2'] = blocks['GEO.id2'].map(int)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index(inplace=True)
blocks.head()

,latitude,longitude,urban,landarea,waterarea,blockgroup
GEO.id2,,,,,,
40019426001000,36.915534,-109.759147,R,11931369,0,40019426001
40019426001001,36.845741,-109.787031,R,72198969,3218,40019426001
40019426001002,36.812747,-109.784069,R,62351,0,40019426001
40019426001003,36.847153,-109.834610,R,12691,0,40019426001
40019426001004,36.858668,-109.824409,R,662399,0,40019426001


In [39]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241666 entries, 40019426001000 to 40279800061142
Data columns (total 6 columns):
latitude      241666 non-null float64
longitude     241666 non-null float64
urban         241666 non-null object
landarea      241666 non-null int64
waterarea     241666 non-null int64
blockgroup    241666 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 12.9+ MB


In [40]:
blocks = blocks.join(populations[['income','blockgrouppop','population']], how='inner')
blocks.shape

(241666, 9)

In [41]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241666 entries, 40019426001000 to 40279800061142
Data columns (total 9 columns):
latitude         241666 non-null float64
longitude        241666 non-null float64
urban            241666 non-null object
landarea         241666 non-null int64
waterarea        241666 non-null int64
blockgroup       241666 non-null int64
income           241666 non-null float64
blockgrouppop    241666 non-null object
population       241666 non-null int64
dtypes: float64(3), int64(4), object(2)
memory usage: 18.4+ MB


In [42]:
blocksdropna(blocks)
blocks.shape

(40486, 9)

In [43]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,latitude,longitude,urban,landarea,waterarea,blockgroup,income,blockgrouppop,population
GEO.id2,,,,,,,,,


In [44]:
testlat, testlong  # used as center of Las Vegas

(33.444506, -112.043629)

In [45]:
blocks['distance_from_city'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [46]:
blocks = blocks[blocks['distance_from_city'] < (maxradius - 1000)]
blocks.shape

(2905, 10)

In [47]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
40131075001008,7928.005497,U,33.513317,-112.066716,40131075001,914.0,62831,41,31165,0
40131075001009,7869.999940,U,33.512366,-112.068371,40131075001,914.0,62831,78,42086,0
40131075001010,7990.910239,U,33.512483,-112.072116,40131075001,914.0,62831,34,29039,0
40131075001011,7906.213443,U,33.511605,-112.072344,40131075001,914.0,62831,26,25054,0
40131075001012,7785.779121,U,33.511525,-112.068550,40131075001,914.0,62831,75,39960,0


In [48]:
blocks[blocks['distance_from_city'] < (maxradius - 1000)][['population']].sum()

population    294309
dtype: int64

### To Skip Over Raw Data Formatting to Saved Train Row Coordinates csv

In [27]:
blocks = pd.read_csv('LasVegasTrainRowCoords.csv',
          index_col='GEO.id2')

blocks.sort_index(inplace=True)
blocks.head()

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block
GEO.id2,,,,,,,,,,,
3.200300e+14,5697.443771,U,36.164760,-115.224611,320030001032,19251,1086,181,51061,0,2000
3.200300e+14,5756.018860,U,36.164769,-115.225839,320030001032,19251,1086,118,27560,0,2001
3.200300e+14,5798.254267,U,36.164774,-115.226710,320030001032,19251,1086,89,28085,0,2002
3.200300e+14,5842.585572,U,36.164785,-115.227595,320030001032,19251,1086,127,28444,0,2003
3.200300e+14,5978.191341,U,36.164629,-115.230546,320030001032,19251,1086,182,34446,0,2006


In [29]:
blocks.duplicated().sum()

553

In [30]:
blocks = blocks.drop_duplicates(keep='first')
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1231 entries, 320030001032000.0 to 320030068004082.0
Data columns (total 11 columns):
distance         1231 non-null float64
urban            1231 non-null object
latitude         1231 non-null float64
longitude        1231 non-null float64
blockgroup       1231 non-null int64
income           1231 non-null int64
blockgrouppop    1231 non-null int64
population       1231 non-null int64
landarea         1231 non-null int64
waterarea        1231 non-null int64
block            1231 non-null int64
dtypes: float64(3), int64(7), object(1)
memory usage: 115.4+ KB


In [31]:
def bootstrapBlocks(blocksdf) :
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>100].index :
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 100))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [32]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Float64Index([320030001032000.0, 320030001032001.0, 320030001032002.0,
              320030001032003.0, 320030001032006.0, 320030001032007.0,
              320030001032008.0, 320030001033005.0, 320030001033006.0,
              320030001033008.0,
              ...
              320030068004034.0, 320030068004035.0, 320030068004055.0,
              320030068004057.0, 320030068004059.0, 320030068004060.0,
              320030068004061.0, 320030068004062.0, 320030068004081.0,
              320030068004082.0],
             dtype='float64', name='GEO.id2', length=1231)


(2717, 11)

In [33]:
blocks.to_csv('LasVegasTrainRowCoordsUpdated.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

## Vectorize Businesses and Assemble X_train

### Construct Category Corpus with Aliases

In [44]:
lat1000m = -115.161605-(-115.172713)

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    try :
        idx = pd.IndexSlice
        df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
        distances = pd.Series(index=df.index)
    except :
        df = multiindexdf.loc[:, ['latitude','longitude']]
        distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances

def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
            coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                            coorddf.loc[index,'latitude'], 
                                            coorddf.loc[index,'longitude'])

In [35]:
df = pd.read_csv('LasVegasBusinessesFinalUpdated.csv',sep=';',compression='gzip',
                 index_col=['lat_index.1','long_index.1'])
df.head()

lat_index  long_index  Unnamed: 0  \
lat_index.1 long_index.1                                      
36.058      -115.183435      36.058 -115.183435       18750   
36.059      -115.183514      36.059 -115.183514       15819   
36.060      -115.211913      36.060 -115.211913       13870   
            -115.210210      36.060 -115.210210       13899   
            -115.210210      36.060 -115.210210       13900   

                                                                 alias  \
lat_index.1 long_index.1                                                 
36.058      -115.183435   best-mattress-liquidation-center-las-vegas-4   
36.059      -115.183514                    ies-residential-las-vegas-2   
36.060      -115.211913                            switchnap-las-vegas   
            -115.210210         martin-harris-construction-las-vegas-5   
            -115.210210                    switch-supernap-7-las-vegas   

                          attributes  \
lat_index.1 long_index.1               
36.058      -115.183435          NaN   
36.059      -115.183514          NaN   
36.060      -115.211913          NaN   
            -115.210210          NaN   
            -115.210210          NaN   

                                                                 categories  \
lat_index.1 long_index.1                                                      
36.058      -115.183435   [{'alias': 'mattresses', 'title': 'Mattresses'...   
36.059      -115.183514   [{'alias': 'excavationservices', 'title': 'Exc...   
36.060      -115.211913   [{'alias': 'isps', 'title': 'Internet Service ...   
            -115.210210   [{'alias': 'contractors', 'title': 'Contractor...   
            -115.210210   [{'alias': 'telecommunications', 'title': 'Tel...   

                                                                coordinates  \
lat_index.1 long_index.1                                                      
36.058      -115.183435   {'latitude': 36.05846, 'longitude': -115.1834351}   
36.059      -115.183514   {'latitude': 36.0593128885251, 'longitude': -1...   
36.060      -115.211913   {'latitude': 36.0603091231611, 'longitude': -1...   
            -115.210210     {'latitude': 36.05989, 'longitude': -115.21021}   
            -115.210210     {'latitude': 36.05989, 'longitude': -115.21021}   

                             distance   latitude   longitude main_category  \
lat_index.1 long_index.1                                                     
36.058      -115.183435   6954.047910  36.058460 -115.183435      shopping   
36.059      -115.183514   6859.204503  36.059313 -115.183514  homeservices   
36.060      -115.211913   6941.060362  36.060309 -115.211913  homeservices   
            -115.210210   6948.238755  36.059890 -115.210210  homeservices   
            -115.210210   6948.238755  36.059890 -115.210210  homeservices   

                                                      name  price  rating  \
lat_index.1 long_index.1                                                    
36.058      -115.183435   Best Mattress Liquidation Center      2     4.0   
36.059      -115.183514                    IES Residential      0     1.0   
36.060      -115.211913                          SwitchNAP      0     1.0   
            -115.210210         Martin Harris Construction      0     1.0   
            -115.210210                  Switch SUPERNAP 7      0     5.0   

                          review_count  \
lat_index.1 long_index.1                 
36.058      -115.183435             22   
36.059      -115.183514              1   
36.060      -115.211913              1   
            -115.210210              1   
            -115.210210              1   

                                                             alias_cats  
lat_index.1 long_index.1                                                 
36.058      -115.183435                          2_furniture 2_shopping  
36.059      -115.183514                                  0_homeservices 

In [37]:
blocks = pd.read_csv('LasVegasTrainRowCoordsUpdated.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.head()

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block
3.200300e+14,5697.443771,U,36.164760,-115.224611,320030001032,19251,1086,181,51061,0,2000
3.200300e+14,5756.018860,U,36.164769,-115.225839,320030001032,19251,1086,118,27560,0,2001
3.200300e+14,5798.254267,U,36.164774,-115.226710,320030001032,19251,1086,89,28085,0,2002
3.200300e+14,5842.585572,U,36.164785,-115.227595,320030001032,19251,1086,127,28444,0,2003
3.200300e+14,5978.191341,U,36.164629,-115.230546,320030001032,19251,1086,182,34446,0,2006


In [43]:
blocks_resumed = blocks[blocks['corpus'].isnull()]
blocks_resumed.shape

(1250, 12)

In [38]:
make_X_train(blocks, df)

(324, 2)
(332, 2)
(335, 2)
(334, 2)
(199, 2)
(207, 2)
(223, 2)
(346, 2)
(354, 2)
(401, 2)
(332, 2)
(338, 2)
(348, 2)
(347, 2)
(241, 2)
(232, 2)
(276, 2)
(334, 2)
(349, 2)
(438, 2)
(493, 2)
(482, 2)
(506, 2)
(474, 2)
(546, 2)
(476, 2)
(466, 2)
(504, 2)
(524, 2)
(514, 2)
(367, 2)
(556, 2)
(574, 2)
(567, 2)
(606, 2)
(595, 2)
(629, 2)
(581, 2)
(563, 2)
(628, 2)
(476, 2)
(466, 2)
(461, 2)
(205, 2)
(420, 2)
(307, 2)
(331, 2)
(285, 2)
(327, 2)
(435, 2)
(269, 2)
(432, 2)
(254, 2)
(275, 2)
(477, 2)
(326, 2)
(341, 2)
(190, 2)
(239, 2)
(194, 2)
(87, 2)
(165, 2)
(158, 2)
(264, 2)
(122, 2)
(283, 2)
(258, 2)
(176, 2)
(168, 2)
(167, 2)
(261, 2)
(286, 2)
(390, 2)
(385, 2)
(406, 2)
(402, 2)
(405, 2)
(290, 2)
(319, 2)
(295, 2)
(423, 2)
(383, 2)
(327, 2)
(289, 2)
(378, 2)
(331, 2)
(311, 2)
(420, 2)
(397, 2)
(393, 2)
(359, 2)
(347, 2)
(328, 2)
(389, 2)
(448, 2)
(437, 2)
(441, 2)
(405, 2)
(33, 2)
(117, 2)
(447, 2)
(766, 2)
(381, 2)
(610, 2)
(192, 2)
(328, 2)
(364, 2)
(939, 2)
(1184, 2)
(1189, 2)
(698, 2)
(

(202, 2)
(162, 2)
(172, 2)
(180, 2)
(186, 2)
(218, 2)
(392, 2)
(373, 2)
(371, 2)
(425, 2)
(419, 2)
(379, 2)
(351, 2)
(365, 2)
(367, 2)
(386, 2)
(333, 2)
(346, 2)
(375, 2)
(420, 2)
(317, 2)
(332, 2)
(219, 2)
(361, 2)
(307, 2)
(143, 2)
(119, 2)
(310, 2)
(180, 2)
(186, 2)
(204, 2)
(170, 2)
(172, 2)
(183, 2)
(437, 2)
(236, 2)
(340, 2)
(346, 2)
(356, 2)
(359, 2)
(409, 2)
(350, 2)
(431, 2)
(430, 2)
(432, 2)
(430, 2)
(361, 2)
(167, 2)
(192, 2)
(205, 2)
(177, 2)
(176, 2)
(400, 2)
(450, 2)
(432, 2)
(228, 2)
(215, 2)
(223, 2)
(429, 2)
(459, 2)
(411, 2)
(57, 2)
(61, 2)
(59, 2)
(64, 2)
(61, 2)
(62, 2)
(83, 2)
(102, 2)
(60, 2)
(63, 2)
(58, 2)
(45, 2)
(1009, 2)
(1179, 2)
(1165, 2)
(1165, 2)
(1934, 2)
(2016, 2)
(1138, 2)
(1952, 2)
(1842, 2)
(887, 2)
(1075, 2)
(1017, 2)
(1298, 2)
(1231, 2)
(1880, 2)
(1066, 2)
(1404, 2)
(961, 2)
(4024, 2)
(1588, 2)
(566, 2)
(573, 2)
(585, 2)
(570, 2)
(590, 2)
(584, 2)
(585, 2)
(572, 2)
(591, 2)
(688, 2)
(599, 2)
(651, 2)
(580, 2)
(589, 2)
(594, 2)
(592, 2)
(255, 2)
(23

TypeError: '9.320030e+15    36.132105
9.320030e+15    36.130851
Name: latitude, dtype: float64' is an invalid key

In [45]:
make_X_train(blocks_resumed, df)

(0, 2)


C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(353, 2)
(353, 2)
(353, 2)
(353, 2)
(353, 2)
(353, 2)
(353, 2)
(353, 2)
(0, 2)
(306, 2)
(312, 2)
(312, 2)
(312, 2)
(312, 2)
(312, 2)
(100, 2)
(89, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(212, 2)
(272, 2)
(276, 2)
(28, 2)
(158, 2)
(161, 2)
(3774, 2)
(3774, 2)
(3774, 2)
(3774, 2)
(3768, 2)
(3768, 2)
(3768, 2)
(3776, 2)
(3776, 2)
(3776, 2)
(3776, 2)
(3736, 2)
(3736, 2)
(3736, 2)
(3281, 2)
(3281, 2)
(3281, 2)
(3281, 2)
(3281, 2)
(3281, 2)
(3281, 2)
(3281, 2)
(3313, 2)
(3313, 2)
(2940, 2)
(751, 2)
(314, 2)
(314, 2)
(314, 2)
(314, 2)
(314, 2)
(878, 2)
(781, 2)
(781, 2)
(781, 2)
(800, 2)
(800, 2)
(800, 2)
(800, 2)
(804, 2)
(772, 2)
(846, 2)
(846, 2)
(1036, 2)
(1036, 2)
(1036, 2)
(1177, 2)
(945, 2)
(964, 2)
(1671, 2)
(1671, 2)
(1671, 2)
(1671, 2)
(1671, 2)
(1671, 2)
(1207, 2)
(1041, 2)
(938, 2)
(938, 2)
(938, 2)
(938, 2)
(938, 2)
(938, 2)
(938, 2)
(938, 2)
(779, 2)
(804, 2)
(800, 2)
(903, 2)
(956, 2)
(1072, 2)
(648, 2)
(648, 2)
(648, 2)
(648, 2)
(64

(572, 2)
(572, 2)
(572, 2)
(572, 2)
(591, 2)
(688, 2)
(651, 2)
(580, 2)
(589, 2)
(589, 2)
(589, 2)
(589, 2)
(589, 2)
(589, 2)
(589, 2)
(589, 2)
(589, 2)
(594, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(592, 2)
(255, 2)
(676, 2)
(657, 2)
(645, 2)
(619, 2)
(577, 2)
(602, 2)
(211, 2)
(211, 2)
(211, 2)
(211, 2)
(211, 2)
(211, 2)
(211, 2)
(712, 2)
(712, 2)
(712, 2)
(712, 2)
(712, 2)
(712, 2)
(845, 2)
(845, 2)
(845, 2)
(709, 2)
(709, 2)
(355, 2)
(1238, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1142, 2)
(1116, 2)
(1116, 2)
(1116, 2)
(1116, 2)
(1116, 2)
(1169, 2)
(258, 2)
(258, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(1241, 2)
(548, 2)
(548, 2)
(548, 2)
(397, 2)
(553, 2)
(553, 2)
(172, 2)
(172, 2)
(172, 2)
(645, 2)
(645, 2)
(645, 2)
(6

In [ ]:
blocks_resumed.info()

In [ ]:
blocks.to_csv('CorpusLasVegasTrainRowCoords.csv',sep=';',compression='gzip')

In [47]:
blocks_resumed.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1250 entries, 9320030019013000.0 to 1320030068004061.0
Data columns (total 12 columns):
distance         1250 non-null float64
urban            1250 non-null object
latitude         1250 non-null float64
longitude        1250 non-null float64
blockgroup       1250 non-null int64
income           1250 non-null int64
blockgrouppop    1250 non-null int64
population       1250 non-null int64
landarea         1250 non-null int64
waterarea        1250 non-null int64
block            1250 non-null int64
corpus           1250 non-null object
dtypes: float64(3), int64(7), object(2)
memory usage: 167.0+ KB


In [50]:
test[test['corpus'].isnull()].shape

(1250, 12)

In [58]:
len(set(blocks_resumed.index))

1248

In [66]:
blocks_resumed.groupby(level=0).filter(lambda x: len(x) > 1)

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block,corpus


In [63]:
dupindex = blocks_resumed.groupby(level=0).filter(lambda x: len(x) > 1).index
blocks_resumed.loc[dupindex].reset_index(drop=True)

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block,corpus
0,5842.777785,U,36.143213,-115.133313,320030019013,22616,1651,950,165249,0,3000,distances alias_cats
1,5799.620254,U,36.141959,-115.133133,320030019013,22616,1651,972,75278,0,3001,distances alias_cats
2,5842.777785,U,36.143213,-115.133313,320030019013,22616,1651,950,165249,0,3000,distances alias_cats
3,5799.620254,U,36.141959,-115.133133,320030019013,22616,1651,972,75278,0,3001,distances alias_cats
4,5104.404502,U,36.119001,-115.135359,320030025012,16732,3958,777,117711,0,2001,distances alias_cats
5,5130.671171,U,36.116226,-115.135300,320030025012,16732,3958,783,145595,0,2002,distances alias_cats
6,5104.404502,U,36.119001,-115.135359,320030025012,16732,3958,777,117711,0,2001,distances alias_cats
7,5130.671171,U,36.116226,-115.135300,320030025012,16732,3958,783,145595,0,2002,distances alias_cats


In [64]:
blocks_resumed = blocks_resumed.append(blocks_resumed.loc[dupindex].reset_index(drop=True))
blocks_resumed.shape

(1258, 12)

In [65]:
blocks_resumed.drop(dupindex, axis=0, inplace=True)
blocks_resumed.shape

(1254, 12)

In [67]:
test = blocks.copy()
test.update(blocks_resumed[['corpus']], overwrite=False)
test.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2717 entries, 320030001032000.0 to 1320030068004061.0
Data columns (total 12 columns):
distance         2717 non-null float64
urban            2717 non-null object
latitude         2717 non-null float64
longitude        2717 non-null float64
blockgroup       2717 non-null int64
income           2717 non-null int64
blockgrouppop    2717 non-null int64
population       2717 non-null int64
landarea         2717 non-null int64
waterarea        2717 non-null int64
block            2717 non-null int64
corpus           2713 non-null object
dtypes: float64(3), int64(7), object(2)
memory usage: 275.9+ KB


In [72]:
test.dropna(inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2713 entries, 320030001032000.0 to 1320030068004061.0
Data columns (total 12 columns):
distance         2713 non-null float64
urban            2713 non-null object
latitude         2713 non-null float64
longitude        2713 non-null float64
blockgroup       2713 non-null int64
income           2713 non-null int64
blockgrouppop    2713 non-null int64
population       2713 non-null int64
landarea         2713 non-null int64
waterarea        2713 non-null int64
block            2713 non-null int64
corpus           2713 non-null object
dtypes: float64(3), int64(7), object(2)
memory usage: 275.5+ KB


In [75]:
test[test['corpus']==('distances alias_cats')] 
# no rows with no businesses within radius, which in error outputs empty dataframe's column labels:
# 'distances alias_cats'

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block,corpus


In [76]:
X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(test['corpus'])

In [77]:
dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)
dfvect.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
0,0,0,0,0,0,9,0,2,0,1,0,0,0,16,0,1,0,8,0,1,0,1,0,9,0,7,0,2,0,0,0,0,0,6,0,6,0,14,0,9,0,0,4,32,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,10,0,2,0,1,0,0,0,15,0,1,0,8,0,1,0,1,0,9,0,8,0,2,0,0,0,0,0,6,0,8,0,14,0,10,0,0,4,33,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,14,0,2,0,1,0,0,0,15,0,1,0,8,0,2,0,1,0,6,0,8,0,2,0,0,0,0,0,6,0,8,0,14,0,10,0,0,4,32,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,14,0,2,0,1,0,0,0,16,0,1,0,9,0,1,0,1,0,6,0,8,0,2,0,0,0,0,0,6,0,8,0,15,0,10,0,0,4,32,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,10,0,30,0,3,0,1,0,0,0,7,0,1,0,7,0,1,0,1,0,5,0,13,0,0,0,0,0,0,0,6,0,8,0,10,0,7,0,0,1,18,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [80]:
joblib.dump(dfvect, 'X_trainLasVegasUpdated.joblib', compress=9)

['X_trainLasVegasUpdated.joblib']

In [81]:
y_trainLasVegas = test['income']

In [82]:
joblib.dump(y_trainLasVegas, 'y_trainLasVegasUpdated.joblib', compress=9)

['y_trainLasVegasUpdated.joblib']

In [79]:
dfvect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Columns: 1482 entries, landarea to 4_physicaltherapy_far
dtypes: uint16(1482)
memory usage: 7.7 MB
